In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.6 MB/s 
     |████████████████████████████████| 596 kB 53.3 MB/s 
     |████████████████████████████████| 101 kB 11.3 MB/s 
     |████████████████████████████████| 6.6 MB 42.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch 
import numpy as np


In [3]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
bert_tokenizer =BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [4]:
def bert_model(question,context,max_length =500):
  input_id = bert_tokenizer.encode(question,context,max_length =500,truncation = True)
  sep_index = input_id.index(102)
  len_ques = sep_index + 1
  len_context =len(input_id)-len_ques
  segment_id = [0]*len_ques + [1]*len_context

  token = bert_tokenizer.convert_ids_to_tokens(input_id)
  start_token_score = model(torch.tensor([input_id]),token_type_ids = torch.tensor([segment_id]))[0]
  end_token_score = model(torch.tensor([input_id]),token_type_ids = torch.tensor([segment_id]))[1]

  start_token_score = start_token_score.detach().numpy().flatten()
  end_token_score = end_token_score.detach().numpy().flatten()

  answer_start_index = np.argmax(start_token_score)
  answer_end_index = np.argmax(end_token_score)

  start_token_scores = np.round(start_token_score[answer_start_index],2)
  end_token_scores = np.round(end_token_score[answer_end_index],2)

  answer =token[answer_start_index]


  for i in range(answer_start_index +1, answer_end_index + 1):
    if token[i][0:2] =='##' :
      answer = answer + token[i][2:]
    else:
      answer += ' '  + token[i]
  if (answer_start_index ==0) or (start_token_scores < 0 ) or (answer == '[SEP]')or (answer_end_index < answer_start_index):
    answer = 'sorry! no answer found in context/passage'
  return (answer_start_index,answer_end_index,start_token_scores, end_token_scores, answer)

#testing model



In [6]:
#@title Question_answer model { vertical-output: true }
#@markdown ---

context = "" #@param{type:'string'}
Question = "" #@param{type:'string'}

#@markdown ---

_,_,_,_,answer = bert_model(Question,context)

#@markdown Answer: 
print(answer)


hype for everything like someone if the they sattirely said any religious jokes
